In [ ]:
import numpy as np
import pylab
%matplotlib inline
import scipy.ndimage as ndimage
from PIL import ImageDraw, ImageFont, Image
import pickle
import os
import cPickle

In [ ]:
def LoadNoiseKernel():
    NoiseArraysFile=open("Pkls/NoiseArrays.pkl",'rb')
    NoiseArray=cPickle.load(NoiseArraysFile)[:,0:3000]
    NoiseArraysFile.close()
    NoiseFFT=np.fft.rfft2(NoiseArray)
    return NoiseFFT
    
NoiseFFT=LoadNoiseKernel()

def NoiseBlast():
    RandomRephase=np.random.uniform(0,6.28,np.shape(NoiseFFT))
    RandomRephase[0,0]=0   #Don't mess with DC mode
    RandomNoise=np.abs(np.fft.irfft2(NoiseFFT*np.exp(RandomRephase*1j)))
    return RandomNoise

def SampleNoise(RandomNoise,NTimes,Size):
    ReturnImages=[]
    for n in range(0,NTimes):
        SampleAreaX=np.shape(RandomNoise)[0]-Size[0]
        SampleAreaY=np.shape(RandomNoise)[1]-Size[1]
        RX=int(np.random.uniform(0,SampleAreaX))
        RY=int(np.random.uniform(0,SampleAreaY))
        SampleImage=RandomNoise[RX:RX+Size[0],RY:RY+Size[1]]
        ReturnImages.append(SampleImage)
    return ReturnImages

In [ ]:
XSize=118
YSize=230
HowManyToTrain=20000
Unblur=False

In [ ]:
fnts=[]
FontNames=os.listdir("./Fonts")
print FontNames


In [ ]:
imgs=[]
fig, axs = pylab.subplots(10,2, sharex=True, sharey=True,dpi=200)
fig.sharex=True
fig.sharey=True
Div=10
for f in range(0,len(FontNames)):
    fnt = ImageFont.truetype('./Fonts/'+str(FontNames[f]), 100)
    img = Image.new('F', (1500,150))
    d = ImageDraw.Draw(img)
    d.text((5, 15), FontNames[f], fill=225,font=fnt)
    axs[f%Div,(f-(f%Div))/Div].imshow(img)
    axs[f%Div,(f-(f%Div))/Div].set_xticks([])
    axs[f%Div,(f-(f%Div))/Div].set_yticks([])


img = Image.new('F', (1500,150))
d = ImageDraw.Draw(img)
d.text((5, 15),"", fill=225,font=fnt)
axs[9,1].imshow(img)
axs[9,1].set_xticks([])
axs[9,1].set_yticks([])




fig.subplots_adjust(hspace=-0.7)   
fig.subplots_adjust(wspace=0)    
pylab.show()




In [ ]:
f

In [ ]:
def MakeTrainingImageBlurred(Letter, XSize, YSize, Rotation, Size, Blur,Noise,ShowIt=True):

    f=np.random.randint(len(FontNames))
    fnt = ImageFont.truetype('./Fonts/'+str(FontNames[f]), Size)
    img = Image.new('F', (XSize,YSize))
    d = ImageDraw.Draw(img)
    X=5+np.random.random()*15
    Y=-15+np.random.random()*20
    d.text((X, Y), Letter, fill=125,font=fnt)
    Rotated=img.rotate(Rotation)
    Blurred1=ndimage.gaussian_filter(Rotated, sigma=(Blur), order=0)
#    img2=Blurred1
    img2=Blurred1+Noise

    Blurred=ndimage.gaussian_filter(img2, sigma=0, order=0)


    Final=Blurred

    MaxVal=np.max(Final)
    MinVal=np.min(Final)

    Final=(Final-MinVal)/(MaxVal-MinVal)
#    for x in range(len(Final)-20,len(Final)):
#        for y in range(len(Final[0])-20,len(Final[0])):
#            Final[x][y]=0   
         #   print(x,y)
    if(ShowIt):
        pylab.imshow(Final,cmap='binary')
        pylab.clim(0,1)
        pylab.show()
    return Final


In [ ]:
XSize

In [ ]:
def LabelToLetter(l):
    return chr(65+l)

In [ ]:
NoiseSamples=[]
HowManyPerBlast=30
for i in range(HowManyToTrain/HowManyPerBlast):
    Samples=SampleNoise(NoiseBlast(),HowManyPerBlast,(YSize,XSize))
    for sample in Samples:
        NoiseSamples.append(sample)
   # print(i*HowManyPerBlast)

In [ ]:
NoiseID=int(np.random.uniform(len(NoiseSamples)))
Img=MakeTrainingImageBlurred("C", XSize, YSize, -5, 155, 10,NoiseSamples[NoiseID])

In [ ]:
pylab.imshow(NoiseSamples[NoiseID],cmap='binary')
pylab.clim(0,255)

In [ ]:
NoiseSamples[NoiseID]

In [ ]:
ShowEvery=100
Labels=[]
Images=[]


NumberToMake=HowManyToTrain
for i in range(0,NumberToMake):

    l=np.random.randint(26)
    Letter=LabelToLetter(l)
    RandomRotation=(np.random.random()-0.5)*2*5
    RandomSize=int(145+20*np.random.random())
    RandomBlur=5+np.random.random()*10

    if(Unblur):
        RandomBlur=0
   # RandomBlur=10
    
    Labels.append(l)
    NoiseID=int(np.random.uniform(len(NoiseSamples)))
    Images.append(MakeTrainingImageBlurred(Letter, XSize, YSize, RandomRotation, RandomSize, RandomBlur,NoiseSamples[NoiseID],False))
    
    if(i%ShowEvery==0):
        print i
        pylab.imshow(Images[-1],cmap='binary')
        pylab.title(Letter)
        pylab.show()



In [ ]:
TestLabels=[]
TestImages=[]
NumTestImages=2000
for i in range(0, NumTestImages):
    if(i%100==0):
        print i
    l=np.random.randint(26)
    Letter=LabelToLetter(l)
    RandomRotation=(np.random.random()-0.5)*2*5
    RandomSize=int(145+20*np.random.random())
    RandomBlur=5+np.random.random()*10
    if(Unblur):
        RandomBlur=0
    TestLabels.append(l)
    NoiseID=int(np.random.uniform(len(NoiseSamples)))

    TestImages.append(MakeTrainingImageBlurred(Letter, XSize, YSize, RandomRotation, RandomSize, RandomBlur,NoiseSamples[NoiseID],False))

In [ ]:
trainingfile=open("Pkls/TrainingFile.pkl",'wb')
pickle.dump(((np.array(Images),np.array(Labels)),(np.array(TestImages),np.array(TestLabels))),trainingfile)
trainingfile.close()

In [ ]:
pylab.figure(figsize=(7,10))
for i in range(25):
    pylab.subplot(5,5,i+1)
    pylab.xticks([])
    pylab.yticks([])
    pylab.grid('off')
    pylab.imshow(Images[i], cmap='binary')
    pylab.xlabel(LabelToLetter(Labels[i]))

In [ ]:
pylab.figure(figsize=(7,10))
for i in range(25):
    pylab.subplot(5,5,i+1)
    pylab.xticks([])
    pylab.yticks([])
    pylab.grid('off')
    pylab.imshow(TestImages[i], cmap='binary')
    pylab.xlabel(LabelToLetter(TestLabels[i]))

In [ ]:
pylab.hist(np.ndarray.flatten(TestImages[4]),bins=np.arange(0,1,0.01))
